# HOW-TO-TWIN-ORCHESTRATION

Quest: Twin Blade
Belt target: Green -> Black

Objective:
- demonstrate CPU prepass + LLM fallback
- verify receipts for every turn
- confirm extension-aware skills/persona/identity loading

Gate:
- CPU routes work (/skills, /recipes, /kernel, /status)
- LLM fallback works with reachable endpoint
- artifacts/twin contains route + graph + hash


In [ ]:
from pathlib import Path
import json
import subprocess

repo = Path.cwd().resolve()
while repo != repo.parent and not (repo / 'cli' / 'src').exists():
    repo = repo.parent
assert (repo / 'cli' / 'src').exists(), 'Repo root not found'

env_prefix = 'PYTHONPATH=' + str(repo / 'src/cli/src')

cpu_cmd = env_prefix + ' python -m stillwater twin "/skills" --json'
print('$ ' + cpu_cmd)
cpu = subprocess.run(['bash', '-lc', cpu_cmd], cwd=str(repo), capture_output=True, text=True)
print(cpu.stdout)
if cpu.returncode != 0:
    print(cpu.stderr)
    raise RuntimeError('CPU twin command failed: ' + str(cpu.returncode))

probe_cmd = env_prefix + ' python -m stillwater llm probe-ollama --json'
probe = subprocess.run(['bash', '-lc', probe_cmd], cwd=str(repo), capture_output=True, text=True)
if probe.returncode != 0:
    print(probe.stdout)
    print(probe.stderr)
    raise RuntimeError('ollama probe failed: ' + str(probe.returncode))
probe_data = json.loads(probe.stdout)
preferred = probe_data.get('preferred')
if preferred:
    llm_cmd = env_prefix + ' python -m stillwater twin "Summarize twin orchestration in 3 bullets." --json'
    print('$ ' + llm_cmd)
    llm = subprocess.run(['bash', '-lc', llm_cmd], cwd=str(repo), capture_output=True, text=True)
    print(llm.stdout)
    if llm.returncode != 0:
        print(llm.stderr)
        raise RuntimeError('LLM twin command failed: ' + str(llm.returncode))
else:
    print('No reachable Ollama endpoint. LLM fallback step skipped.')